# Imports

In [223]:
import os
import re

# pandas
import pandas as pd, numpy as np
from pandas import DataFrame

# tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

# cross-validation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ML classifiers
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

# Sentiment Analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from scipy.sparse import hstack

# Read data

In [224]:
DATA_PATH = '/Users/cep215/Desktop/book-reviews/jupyter/exports/'

def read_data(data_path = DATA_PATH):
    data_path = os.path.join(data_path, 'parent-child.csv')
    df = pd.read_csv(data_path)
    return df
data = read_data()
pairs_df = pd.DataFrame(data)
data.loc[pairs_df["Relationship"]==True,"Relationship",]=1
data.loc[pairs_df["Relationship"]==False,"Relationship",]=0

# Summary

In [225]:
data.describe().transpose()

count      mean       std  min  25%  50%  75%  max
Level         487.0  2.412731  0.829052  2.0  2.0  2.0  3.0  8.0
Relationship  487.0  0.439425  0.496827  0.0  0.0  0.0  1.0  1.0
Supports      487.0  0.063655  0.387635  0.0  0.0  0.0  0.0  4.0
Attacks       487.0  0.004107  0.090629  0.0  0.0  0.0  0.0  2.0
Neutrals      487.0  0.002053  0.045314  0.0  0.0  0.0  0.0  1.0

# Data (DF_X) & Label(DF_Y)

In [226]:
pairs = []
sent_diff = []
for index, row in pairs_df.iterrows():
    pair = row["Parent"] + " STOP " + row["Post"]
    pairs.append(pair)
    analyzer = SentimentIntensityAnalyzer()
    sent_child = analyzer.polarity_scores(row["Parent"])['compound']
    sent_parent = analyzer.polarity_scores(row["Post"])['compound']
    if (sent_child < sent_parent):
        sent_diff.append(sent_parent - sent_child)
    else:
        sent_diff.append(sent_child - sent_parent)
df_x = pd.DataFrame(pairs)[0]
df_y=pairs_df["Relationship"]
sent_diff = pd.DataFrame(sent_diff)

# Add TF-IFD feature

In [227]:
tfidf = TfidfVectorizer(min_df=1)
df_x = tfidf.fit_transform(df_x)

# Add Sentiment Feature

In [236]:
df_x = hstack((df_x , np.array(sent_diff[0])[:,None]))
df_x = df_x.tocsr()

# Cross-Validation

In [228]:
skf = StratifiedKFold(n_splits=8)

# Decision Tree

In [229]:
acc = 0
f1 = 0
for train_index, test_index in skf.split(df_x, df_y):
    x_train, x_test = df_x[train_index], df_x[test_index]
    y_train, y_test = df_y[train_index], df_y[test_index]
    clf = DecisionTreeClassifier()
    clf.fit(x_train, y_train) 
    prediction = clf.predict(x_test)
    print(classification_report(y_test, prediction))
    f1 += f1_score(y_test, prediction, average='weighted')
    acc += accuracy_score(y_test, prediction)
    
print (acc/8)
print(f1/8)

             precision    recall  f1-score   support

          0       0.61      0.63      0.62        35
          1       0.50      0.48      0.49        27

avg / total       0.56      0.56      0.56        62

             precision    recall  f1-score   support

          0       0.59      0.71      0.64        34
          1       0.50      0.37      0.43        27

avg / total       0.55      0.56      0.55        61

             precision    recall  f1-score   support

          0       0.52      0.41      0.46        34
          1       0.41      0.52      0.46        27

avg / total       0.47      0.46      0.46        61

             precision    recall  f1-score   support

          0       0.59      0.68      0.63        34
          1       0.50      0.41      0.45        27

avg / total       0.55      0.56      0.55        61

             precision    recall  f1-score   support

          0       0.53      0.59      0.56        34
          1       0.39      0.33 

# Random Forest

In [230]:
acc = 0
f1 = 0
for train_index, test_index in skf.split(df_x, df_y):
    x_train, x_test = df_x[train_index], df_x[test_index]
    y_train, y_test = df_y[train_index], df_y[test_index]
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train) 
    prediction = clf.predict(x_test)
    print(classification_report(y_test, prediction))
    f1 += f1_score(y_test, prediction, average='weighted')
    acc += accuracy_score(y_test, prediction)
    
print (acc/8)
print(f1/8)

             precision    recall  f1-score   support

          0       0.62      0.83      0.71        35
          1       0.60      0.33      0.43        27

avg / total       0.61      0.61      0.59        62

             precision    recall  f1-score   support

          0       0.55      0.82      0.66        34
          1       0.40      0.15      0.22        27

avg / total       0.48      0.52      0.46        61

             precision    recall  f1-score   support

          0       0.61      0.82      0.70        34
          1       0.60      0.33      0.43        27

avg / total       0.60      0.61      0.58        61

             precision    recall  f1-score   support

          0       0.50      0.74      0.60        34
          1       0.18      0.07      0.11        27

avg / total       0.36      0.44      0.38        61

             precision    recall  f1-score   support

          0       0.55      0.68      0.61        34
          1       0.42      0.30 

# LinearSVC

In [231]:
acc = 0
f1 = 0
for train_index, test_index in skf.split(df_x, df_y):
    x_train, x_test = df_x[train_index], df_x[test_index]
    y_train, y_test = df_y[train_index], df_y[test_index]
    clf = LinearSVC()
    clf.fit(x_train, y_train) 
    prediction = clf.predict(x_test)
    print(classification_report(y_test, prediction))
    f1 += f1_score(y_test, prediction, average='weighted')
    acc += accuracy_score(y_test, prediction)
print (acc/8)
print(f1/8)

             precision    recall  f1-score   support

          0       0.64      0.66      0.65        35
          1       0.54      0.52      0.53        27

avg / total       0.60      0.60      0.60        62

             precision    recall  f1-score   support

          0       0.59      0.79      0.68        34
          1       0.53      0.30      0.38        27

avg / total       0.56      0.57      0.54        61

             precision    recall  f1-score   support

          0       0.56      0.65      0.60        34
          1       0.45      0.37      0.41        27

avg / total       0.52      0.52      0.52        61

             precision    recall  f1-score   support

          0       0.49      0.56      0.52        34
          1       0.32      0.26      0.29        27

avg / total       0.41      0.43      0.42        61

             precision    recall  f1-score   support

          0       0.35      0.26      0.30        34
          1       0.29      0.37 

# NuSVC

In [232]:
acc = 0
f1 = 0
for train_index, test_index in skf.split(df_x, df_y):
    x_train, x_test = df_x[train_index], df_x[test_index]
    y_train, y_test = df_y[train_index], df_y[test_index]
    clf = NuSVC()
    clf.fit(x_train, y_train) 
    prediction = clf.predict(x_test)
    print(classification_report(y_test, prediction))
    f1 += f1_score(y_test, prediction, average='weighted')
    acc += accuracy_score(y_test, prediction)
print (acc/8)
print(f1/8)

             precision    recall  f1-score   support

          0       0.61      0.77      0.68        35
          1       0.56      0.37      0.44        27

avg / total       0.59      0.60      0.58        62

             precision    recall  f1-score   support

          0       0.59      0.76      0.67        34
          1       0.53      0.33      0.41        27

avg / total       0.56      0.57      0.55        61

             precision    recall  f1-score   support

          0       0.59      0.56      0.58        34
          1       0.48      0.52      0.50        27

avg / total       0.54      0.54      0.54        61

             precision    recall  f1-score   support

          0       0.52      0.50      0.51        34
          1       0.39      0.41      0.40        27

avg / total       0.46      0.46      0.46        61

             precision    recall  f1-score   support

          0       0.29      0.18      0.22        34
          1       0.30      0.44 

# Neural Network

In [233]:
acc = 0
f1 = 0
for train_index, test_index in skf.split(df_x, df_y):
    x_train, x_test = df_x[train_index], df_x[test_index]
    y_train, y_test = df_y[train_index], df_y[test_index]
    clf = MLPClassifier(activation = 'logistic', learning_rate='adaptive', solver='lbfgs')
    clf.fit(x_train, y_train) 
    prediction = clf.predict(x_test)
    print(classification_report(y_test, prediction))
    avg += f1_score(y_test, prediction, average='weighted')
    acc += accuracy_score(y_test, prediction)
print (acc/8)
print(f1/8)

             precision    recall  f1-score   support

          0       0.62      0.71      0.67        35
          1       0.55      0.44      0.49        27

avg / total       0.59      0.60      0.59        62

             precision    recall  f1-score   support

          0       0.64      0.82      0.72        34
          1       0.65      0.41      0.50        27

avg / total       0.64      0.64      0.62        61

             precision    recall  f1-score   support

          0       0.59      0.65      0.62        34
          1       0.50      0.44      0.47        27

avg / total       0.55      0.56      0.55        61

             precision    recall  f1-score   support

          0       0.51      0.59      0.55        34
          1       0.36      0.30      0.33        27

avg / total       0.45      0.46      0.45        61

             precision    recall  f1-score   support

          0       0.43      0.29      0.35        34
          1       0.37      0.52 

# Nearest Neighbour

In [234]:
acc = 0
f1 = 0
for train_index, test_index in skf.split(df_x, df_y):
    x_train, x_test = df_x[train_index], df_x[test_index]
    y_train, y_test = df_y[train_index], df_y[test_index]
    clf = KNeighborsClassifier()
    clf.fit(x_train, y_train) 
    prediction = clf.predict(x_test)
    print(classification_report(y_test, prediction))
    f1 += f1_score(y_test, prediction, average='weighted')
    acc += accuracy_score(y_test, prediction)
print (acc/8)
print(f1/8)

             precision    recall  f1-score   support

          0       0.56      0.66      0.61        35
          1       0.43      0.33      0.38        27

avg / total       0.50      0.52      0.50        62

             precision    recall  f1-score   support

          0       0.63      0.71      0.67        34
          1       0.57      0.48      0.52        27

avg / total       0.60      0.61      0.60        61

             precision    recall  f1-score   support

          0       0.50      0.56      0.53        34
          1       0.35      0.30      0.32        27

avg / total       0.43      0.44      0.44        61

             precision    recall  f1-score   support

          0       0.51      0.71      0.59        34
          1       0.29      0.15      0.20        27

avg / total       0.41      0.46      0.42        61

             precision    recall  f1-score   support

          0       0.54      0.62      0.58        34
          1       0.41      0.33 

# Naive Bayesian

In [235]:
acc = 0
f1 = 0
for train_index, test_index in skf.split(df_x, df_y):
    x_train, x_test = df_x[train_index], df_x[test_index]
    y_train, y_test = df_y[train_index], df_y[test_index]
    clf = MultinomialNB()
    clf.fit(x_train, y_train) 
    prediction = clf.predict(x_test)
    print(classification_report(y_test, prediction))
    f1 += f1_score(y_test, prediction, average='weighted')
    acc += accuracy_score(y_test, prediction)
print (acc/8)
print(f1/8)

             precision    recall  f1-score   support

          0       0.57      0.94      0.71        35
          1       0.50      0.07      0.13        27

avg / total       0.54      0.56      0.46        62

             precision    recall  f1-score   support

          0       0.60      1.00      0.75        34
          1       1.00      0.15      0.26        27

avg / total       0.78      0.62      0.53        61

             precision    recall  f1-score   support

          0       0.55      0.85      0.67        34
          1       0.38      0.11      0.17        27

avg / total       0.47      0.52      0.45        61

             precision    recall  f1-score   support

          0       0.51      0.79      0.62        34
          1       0.12      0.04      0.06        27

avg / total       0.34      0.46      0.37        61

             precision    recall  f1-score   support

          0       0.54      0.82      0.65        34
          1       0.33      0.11 